In [1]:
import numpy as np

Введём начальные данные, т.е. последовательность наблюдений o, начальные вероятности для скрытых состояний pi, матрицу переходных состояний P, длину последовательности N, количество скрытых состояний k, матрицу B распределений видимых состояний для всех скрытых состояний:

In [2]:
o = np.array([1,3,3,2,3,3,3,2,1,2,1,1,3,2,3,2,2,1,3,3,1,1,1,2,1])

In [3]:
o -= 1
o

array([0, 2, 2, 1, 2, 2, 2, 1, 0, 1, 0, 0, 2, 1, 2, 1, 1, 0, 2, 2, 0, 0,
       0, 1, 0])

In [4]:
pi = np.array([0.8, 0.2])

In [5]:
P = np.array([[0.6, 0.4], [0.5, 0.5]])

In [6]:
N = len(o)
k = 2

In [7]:
B = np.array([[0.2, 0.4, 0.4], [0.5, 0.4, 0.1]])

Для решения первой части задачи применим алгоритм прямого прохода, для него введём вспомогательный массив alpha и последовательно найдём значения в нём:

In [8]:
alpha = np.zeros((N, k))

In [9]:
for j in range(k):
    alpha[0, j] = B[j, o[0]] * pi[j]
for i in range(1, N):
    for j in range(k):
        alpha[i, j] = B[j, o[i]] * (P[0, j] * alpha[i - 1, 0] + P[1, j] * alpha[i - 1, 1])

Ответом будет сумма чисел в alpha в последний момент времени по всем состояниям:

In [10]:
alpha[N - 1, 0] + alpha[N - 1, 1]

4.978300204739012e-13

Для второй части задачи применим алгоритм Витерби, введём вспомогательный массив beta и прямым проходом его последовательно заполним:

In [11]:
beta = np.zeros((N, k))
sequence = np.zeros(N).astype(int)

In [12]:
for j in range(k):
    beta[0, j] = B[j, o[0]] * pi[j]
for i in range(1, N):
    for j in range(k):
        beta[i, j] = B[j, o[i]] * max(beta[i - 1, 0] * P[0, j], beta[i - 1, 1] * P[1, j])

Для восстановления наиболее вероятной последовательности скрытых состояний применим обратный проход:

In [13]:
sequence = np.zeros(N).astype(int)

In [14]:
sequence[-1] = np.argmax(beta[-1])
for i in range(N - 2, -1, -1):
    sequence[i] = np.argmax([beta[i, 0] * P[0, sequence[i + 1]], beta[i, 1] * P[1, sequence[i + 1]]])

И запишем ответ в терминах задачи ('HOT', 'COLD'):

In [15]:
sequence = np.where(sequence == 0, 'HOT', 'COLD')

In [16]:
sequence

array(['HOT', 'HOT', 'HOT', 'HOT', 'HOT', 'HOT', 'HOT', 'HOT', 'COLD',
       'COLD', 'COLD', 'COLD', 'HOT', 'HOT', 'HOT', 'HOT', 'HOT', 'COLD',
       'HOT', 'HOT', 'COLD', 'COLD', 'COLD', 'COLD', 'COLD'], dtype='<U4')